# Random Forest Model

In [ ]:
import utils
import models.categorical
import math
import pickle
import numpy as np
import pandas as pd
from datetime import date

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

# Hyperparameter Tuning

**Parameters**
- `n_estimators` (default=100): Number of trees in the forest.
- `criterion` (default='gini'): Function to measure quality of split. Either Gini coefficient ('gini') or information gain ('entropy').
- `max_depth` (default=None): If None, nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.
- `min_samples_split` (default=2)
- `min_samples_leaf` (default=1)

**Inputs / X_vars combinations**
- `pct`: `['CLOSE_PCT', 'VOL_PCT']`
- `pct_tech`: `['CLOSE_PCT', 'VOL_PCT', 'MACD', 'RSI14', 'VPT']`
- `pct_macro`: `['CLOSE_PCT', 'VOL_PCT', +macroIndicators]`
- `pct_tech_macro`: `['CLOSE_PCT', 'VOL_PCT', 'MACD', 'RSI14', 'VPT', +macroIndicators]`
    - Note that macroeconomic indicators are added depending on the industry of the future, as well as the country of origin of the future (only US futures have +macroIndicators, as the indicators are USA-based)

In [ ]:
rfParams = [
    {
        "n_estimators": [100], 
        "criterion": ['gini'],
        # "max_depth": [5, 10, 15],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 5, 10, 15],
        # "max_features": ['sqrt', 'log2']
    }
]


start_date = date(2010, 1, 1)
end_date = date(2021, 1, 1)

In [ ]:
# retrieve parameter grid
parameter_grid = list(ParameterGrid(rfParams))
y_var = "LONG_SHORT" # DO NOT CHANGE THIS
file_dir = "rf/pct_tech_macro/" # pct / pct_tech / pct_macro / pct_tech_macro

for future in tqdm(utils.futuresList):
    print(future)

    # load data - generates df with PCT and DIFF of close, tech indicators, and macro indicators
    df = utils.prepare_data(future)
    
    # generate X vars
    # tech indicators ["MACD", "RSI14", "VPT"]
    X_vars = utils.generate_X_vars(future, linearise=False, \
                                   tech_indicators=["MACD", "RSI14", "VPT"], \
                                   macro_indicators=True)
    
    print(X_vars)
        
    # if 'CLOSE_LINEAR_PCT' in X_vars: # only 10 futures models require linearisation

    # load X and y
    X_df, y_df = utils.generate_X_y(df, X_vars=X_vars, y_var=y_var)
    cost_df = df["CLOSE_PCT"]

    # prepare collated results
    agg_results_collated = pd.DataFrame(index=list(range(len(parameter_grid))), 
                                        columns=["accuracy_SMA", "opp_cost_SMA"])
    win_results_collated = []

    # run walk forward validation 
    for i in range(len(parameter_grid)):
        param_set = parameter_grid[i]
        model = RandomForestClassifier(**param_set)
        win_results, agg_results = models.categorical.walk_forward(
            model = model, X = X_df, y = y_df, cost_weight = cost_df, rolling = True, 
            max_windows = 100, start_index = start_date
        )
        win_results_collated.append(win_results)
        agg_results_collated.loc[i, "accuracy_SMA"] = agg_results.loc["SMA", "accuracy"]
        agg_results_collated.loc[i, "opp_cost_SMA"] = agg_results.loc["SMA", "opp_cost"]

    # save parameters
    parameter_df = pd.DataFrame.from_records(parameter_grid)
    combined_df = pd.concat([parameter_df, agg_results_collated], axis=1)
    # sort by lowest opp cost
    combined_df = combined_df.sort_values(by=["opp_cost_SMA"], ascending=True)
    combined_df.to_csv(f"model_metrics/categorical/{file_dir}{future}.csv", index=False)

# Results of Tuning

## Opp Cost Summary

In [ ]:
filepaths = ['pct', 'pct_tech', 'pct_macro', 'pct_tech_macro']

# get validation statistics
lowest_cost_lst = []

param_df = pd.DataFrame()

for filepath in filepaths:
    print(filepath)
    for future in tqdm(utils.futuresList):
        future_metric = pd.read_csv(f'model_metrics/categorical/rf/{filepath}/{future}.csv')
        # get lowest opp cost
        future_cost = future_metric.loc[future_metric['opp_cost_SMA'] == min(future_metric['opp_cost_SMA'])].reset_index(drop=True)
        future_cost_val = future_cost.iloc[0]['opp_cost_SMA']
        lowest_cost_lst.append(future_cost_val)

        row = dict()

        for col in future_cost.columns:
            if (col[:8] != 'opp_cost') & (col[:8] != 'accuracy'):
                row[col] = future_cost.iloc[0][col]

        row['future'] = future
        param_df = param_df.append(row, ignore_index=True)
    
    print("Average Opp Cost: ", sum(lowest_cost_lst)/len(lowest_cost_lst))
    print(param_df['min_samples_split'].value_counts())
    print(param_df['min_samples_leaf'].value_counts())
    print("=============================================")

## Best X_vars Summary

**Evaluated based on: Lowest Opp Cost SMA**

`[CLOSE_PCT, VOL_PCT]` vs `[CLOSE_PCT, VOL_PCT, MACD, RSI14, VPT]`
- 47 out of 88 futures performed better with added tech indicators

`[CLOSE_PCT, VOL_PCT]` vs `[CLOSE_PCT, VOL_PCT, +macroIndicators]`
- 44 out of 88 futures performed better with added macro indicators

`[CLOSE_PCT, VOL_PCT]` vs `[CLOSE_PCT, VOL_PCT, MACD, RSI14, VPT, +macroIndicators]`
- 47 out of 88 futures performed better with added macro indicators

`[CLOSE_PCT, VOL_PCT, MACD, RSI14, VPT]` vs `[CLOSE_PCT, VOL_PCT, +macroIndicators]`
- 40 out of 88 futures performed better with added macro indicators compared to tech indicators

`[CLOSE_PCT, VOL_PCT, MACD, RSI14, VPT]` vs `[CLOSE_PCT, VOL_PCT, MACD, RSI14, VPT, +macroIndicators]`
- 44 out of 88 futures performed better in the second case

`[CLOSE_PCT, VOL_PCT, +macroIndicators]` vs `[CLOSE_PCT, VOL_PCT, MACD, RSI14, VPT, +macroIndicators]`
- 49 out of 88 futures performed better in the second case

In [ ]:
count = 0

for future in tqdm(utils.futuresList):
    perc = pd.read_csv(f"model_metrics/categorical/rf/pct_macro/{future}.csv")
    perc_best = max(perc['opp_cost_SMA'])
        
    perc_2 = pd.read_csv(f"model_metrics/categorical/rf/pct_tech_macro/{future}.csv")
    perc_2_best = max(perc_2['opp_cost_SMA'])
    if (perc_2_best <= perc_best):
        # second model is better
        count += 1
        print('*', future, perc_best-perc_2_best)
    else:
        print(future, perc_2_best-perc_best)

print(count)

## Best Tuning Params Summary

- `min_samples_split` seems to be a good tuning parameter (tried 2, 5, 10), different models work differently depending on this param
- `min_samples_leaf` (tried 1, 5, 10, 15, default=1), different models work differently depending on this param

In [ ]:
best_params = pd.DataFrame()

for future in tqdm(utils.futuresList):
    perc = pd.read_csv(f"model_metrics/categorical/rf/pct_tech_macro/{future}.csv")
    perc_best = max(perc['opp_cost_SMA'])
    best_params = best_params.append(perc.loc[perc['opp_cost_SMA'] == perc_best], ignore_index=True)

In [ ]:
best_params['min_samples_split'].value_counts()

In [ ]:
best_params['min_samples_leaf'].value_counts()

# Save Meta Predictions

In [ ]:
file_dir = "rf/pct"
y_var = "LONG_SHORT"

for future in tqdm(utils.futuresList):
    print(future)
    
    # generate X vars
    X_vars = utils.generate_X_vars(future, linearise=False, \
                                   tech_indicators=[], macro_indicators=False)
    
    models.categorical.save_meta_predictions(
        path = file_dir, metric="opp_cost_SMA", model_fn = RandomForestClassifier, future = future,
        X_vars = X_vars, y_var = y_var, model_name = "rf"
    )

In [ ]:
file_dir = "rf/pct_tech"
y_var = "LONG_SHORT"

for future in tqdm(utils.futuresList):
    print(future)
    
    # generate X vars
    X_vars = utils.generate_X_vars(future, linearise=False, \
                                   tech_indicators=['MACD', 'RSI14', 'VPT'], macro_indicators=False)
    
    models.categorical.save_meta_predictions(
        path = file_dir, metric="opp_cost_SMA", model_fn = RandomForestClassifier, future = future,
        X_vars = X_vars, y_var = y_var, model_name = "rf"
    )

In [ ]:
file_dir = "rf/pct_macro"
y_var = "LONG_SHORT"

for future in tqdm(utils.futuresList):
    print(future)
    
    # generate X vars
    X_vars = utils.generate_X_vars(future, linearise=False, \
                                   tech_indicators=[], macro_indicators=True)
    
    models.categorical.save_meta_predictions(
        path = file_dir, metric="opp_cost_SMA", model_fn = RandomForestClassifier, future = future,
        X_vars = X_vars, y_var = y_var, model_name = "rf"
    )

In [ ]:
file_dir = "rf/pct_tech_macro"
y_var = "LONG_SHORT"

for future in tqdm(utils.futuresList):
    print(future)
    
    # generate X vars
    X_vars = utils.generate_X_vars(future, linearise=False, \
                                   tech_indicators=['MACD', 'RSI14', 'VPT'], macro_indicators=True)
    
    models.categorical.save_meta_predictions(
        path = file_dir, metric="opp_cost_SMA", model_fn = RandomForestClassifier, future = future,
        X_vars = X_vars, y_var = y_var, model_name = "rf"
    )

# Save Models
- Train: (2018, 9, 1) to (2020, 9, 30)
- Val: (2020, 10, 1) to (2020, 12, 31)

=========================================

- Train: (2019, 1, 1) to (2020, 12, 31)
- Test: (2021, 1, 1) to (2021, 3, 31)

In [ ]:
train_start = date(2018, 9, 1)
train_end = date(2020, 9, 30)

for future in utils.futuresList:
    # prepare X variables
    X_vars = utils.generate_X_vars(future, linearise=False, \
                                   tech_indicators=[], \
                                   macro_indicators=False)
    
    # save model
    models.categorical.save_model(path='rf/pct', metric="opp_cost_SMA", \
                                  model_fn=RandomForestClassifier, model_wrapper=models.categorical.RFWrapper, \
                                  future=future, X_vars=X_vars, y_var="LONG_SHORT", \
                                  ext_path="csv",\
                                  train_start=train_start, train_end=train_end)
    # print(f'{future} done')

In [ ]:
train_start = date(2018, 9, 1)
train_end = date(2020, 9, 30)

for future in utils.futuresList:
    # prepare X variables
    X_vars = utils.generate_X_vars(future, linearise=False, \
                                   tech_indicators=["MACD", "RSI14", "VPT"], \
                                   macro_indicators=False)
    
    # save model
    models.categorical.save_model(path='rf/pct_tech', metric="opp_cost_SMA", \
                                  model_fn=RandomForestClassifier, model_wrapper=models.categorical.RFWrapper, \
                                  future=future, X_vars=X_vars, y_var="LONG_SHORT", \
                                  ext_path="csv",\
                                  train_start=train_start, train_end=train_end)
    # print(f'{future} done')

In [ ]:
train_start = date(2018, 9, 1)
train_end = date(2020, 9, 30)

for future in utils.futuresList:
    # prepare X variables
    X_vars = utils.generate_X_vars(future, linearise=False, \
                                   tech_indicators=[], \
                                   macro_indicators=True)
    
    # save model
    models.categorical.save_model(path='rf/pct_macro', metric="opp_cost_SMA", \
                                  model_fn=RandomForestClassifier, model_wrapper=models.categorical.RFWrapper, \
                                  future=future, X_vars=X_vars, y_var="LONG_SHORT", \
                                  ext_path="csv",\
                                  train_start=train_start, train_end=train_end)
    # print(f'{future} done')

In [ ]:
train_start = date(2018, 9, 1)
train_end = date(2020, 9, 30)

for future in utils.futuresList:
    # prepare X variables
    X_vars = utils.generate_X_vars(future, linearise=False, \
                                   tech_indicators=["MACD", "RSI14", "VPT"], \
                                   macro_indicators=True)
    
    # save model
    models.categorical.save_model(path='rf/pct_tech_macro', metric="opp_cost_SMA", \
                                  model_fn=RandomForestClassifier, model_wrapper=models.categorical.RFWrapper, \
                                  future=future, X_vars=X_vars, y_var="LONG_SHORT", \
                                  ext_path="csv",\
                                  train_start=train_start, train_end=train_end)
    # print(f'{future} done')